In [1]:
import matplotlib.pyplot as plt
# from scipy.fftpack import fft, ifft, dct, idct
from scipy.io import wavfile # get the api
import numpy as np
from IPython.display import Audio
import math

fs, audio = wavfile.read('440_sine.wav')
Audio('440_sine.wav')

In [2]:
print(f"Audio Type: {audio.dtype}")
print(f"Samples = {audio.shape[0]}   Channels = {audio.shape[1]}") #output=(samples,channels)
print(f"Sampling frequency = {fs} Hz")

audio = np.mean(audio, axis=1).astype(np.int16) #mean of 2-channel audio or use audio.T[0] to select channel
# audio = audio / (2**15)
samples = audio.shape[0]
L = (samples / fs)*1000
print(f'Audio length: {L:.0f} mili-seconds')

Audio Type: int16
Samples = 5292   Channels = 2
Sampling frequency = 44100 Hz
Audio length: 120 mili-seconds


In [3]:
def dft(x): #calculating dft for x
    N = len(x)
    D = np.ones((N,N),dtype = complex)
    for i in range(1,N):
        for k in range(1,N):
            D[i][k] = np.exp(-2j*(np.pi*i*k)/N)

    y = np.matmul(D,np.transpose(x))
    return np.transpose(y)

In [4]:
def idft(y): #calculating idft for x
    N = len(y)
    D = np.ones((N,N),dtype = complex)
    for i in range(1,N):
        for k in range(1,N):
            D[i][k] = np.exp(2j*(np.pi*i*k)/N)

    z = np.matmul(D/N,np.transpose(y))
    return np.transpose(z)

In [5]:
def Edft(x,L): #calculating error dft for x
    y = dft(x)
    N = len(y)
    a = int((N+1-L)/2)
    b = int((N-1+L)/2)
    for i in range(a,b+1):
        y[i] = 0
    x_m = idft(y)

    return ((x - x_m) ** 2).mean(axis=0)

In [6]:
def dct(x): #calculating dct for x
    N  = len(x)
    Y = np.array([0.]*N)
    for k in range(N):
        for i in range(N):
            Y[k] += 2.0*x[i]*math.cos(math.pi*k*(2.0*i+1)/(2.0*N))
    return Y

In [7]:
def idct(y): #calculating idct for x
    N  = len(y)
    Y = np.array([0.]*N)
    for i in range(N):
        for k in range(N):
            if k == 0:
                CONST = 0.5
            else:
                CONST = 1    
            Y[i] += CONST*y[k]*math.cos(math.pi*k*(2*i+1)/(2*N))
    return Y/N

In [8]:
def Edct(x,L): #calculating error dct for x
    y = dct(x)
    N = len(y)
    for i in range(N-L,N):
        y[i] = 0
    x_m = idct(y)

    return ((x - x_m) ** 2).mean(axis=0)

In [9]:
h2 = np.array([[1,1],[1,-1]]) #calculating haar transform for x
def haar_mat(n):
    n = int(n)
    if n == 1:
        return h2
    else:
        a = np.kron(haar_mat(n-1),[1,1])
        b = np.kron(np.identity(int(math.pow(2,n-1)))*math.pow(2,(n-1)/2.0),[1,-1])
        #print(np.concatenate((a,b),axis=0))
        return np.concatenate((a,b),axis=0)

def haar(x):
    return np.matmul(haar_mat(math.log(len(x),2)),np.transpose(x))

In [10]:
def ihaar(y): #calculating inverse haar for x
    n = int(math.log(len(y),2))
    N = len(y)
    hn = haar_mat(n)
    return np.matmul(np.transpose(hn)/N,np.transpose(y))

In [11]:
def Ehaar(x,L): #calculating error for haar transform
    y = haar(x)
    N = len(y)
    for i in range(N-L,N):
        y[i] = 0
    x_m = ihaar(y)

    return ((x - x_m) ** 2).mean(axis=0)

In [ ]:
plt.figure()
ydft = [0.]*len(audio)
ydct = [0.]*len(audio)
yhaar = [0.]*len(audio)
for L in range(len(audio)):
    ydft[L] = Edft(audio,L)
    ydct[L] = Edct(audio,L)
    yhaar[L] = Ehaar(audio,L) 
plt.title('Energy Compaction Plots')
plt.plot(range(64),ydft,'--b',label = "dft")
plt.plot(range(64),ydct,'-r',label = 'dct')
plt.plot(range(64),yhaar,'-g',label = 'haar')
plt.legend(loc='upper left')    
plt.xlabel("L")
plt.ylabel("E(L)")
plt.show()